In [ ]:
[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]

In [2]:
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import numpy as np
import json

plt.style.use('seaborn-v0_8-bright')
plt.style.use('ggplot')

o_df = pd.read_csv("tmdb_5000_movies.csv")

o_df.head(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [13]:
o_df.drop(['homepage', 'tagline'], axis=1, inplace=True)
o_df.dropna(inplace=True)

In [14]:
columns_to_remove_zero = ['runtime', 'vote_average','vote_count','popularity']  

for col in columns_to_remove_zero:
    o_df = o_df[o_df[col] != 0]


In [20]:
def extract_ids(json_str):
    genres_list = json.loads(json_str)  # Parse JSON string
    ids = [genre['id'] for genre in genres_list if 'id' in genre]  # Extract IDs
    return ids
def extract_ids2(json_str):
    genres_list = json.loads(json_str)  # Parse JSON string
    ids = [genre['iso_639_1'] for genre in genres_list if 'iso_639_1' in genre]  # Extract IDs
    return ids

o_df['production_companies_id'] = o_df['production_companies'].apply(extract_ids)
o_df['spoken_languages_id'] = o_df['spoken_languages'].apply(extract_ids2)


In [54]:
# Step 1: Split the list of categorical values into separate rows
data_expanded = o_df.explode('production_companies_id')

# Step 2: Create dummy variables for each unique category
dummy_variables = pd.get_dummies(data_expanded['production_companies_id'], prefix='production_companies_id')

# Step 3: Aggregate the dummy variables
aggregated_dummy = dummy_variables.groupby(level=0).sum()

# Concatenate the aggregated dummy variables with the original DataFrame
data_encoded = pd.concat([o_df[['revenue']], aggregated_dummy], axis=1)
# o_df['encoded_production_companies'] = aggregated_dummy.apply(lambda x: np.array(x), axis=1)

data_encoded

,revenue,production_companies_id_1,production_companies_id_2,production_companies_id_3,production_companies_id_4,production_companies_id_5,production_companies_id_6,production_companies_id_7,production_companies_id_8,production_companies_id_9,...,production_companies_id_93528,production_companies_id_93542,production_companies_id_93580,production_companies_id_94410,production_companies_id_94411,production_companies_id_94551,production_companies_id_94552,production_companies_id_94714,production_companies_id_95018,production_companies_id_95063
0,2787965087,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,961000000,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,880674609,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1084939099,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,284139100,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,2040920,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4799,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4800,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4801,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
from scipy.stats import pearsonr
correlations = {}
p_values = {}
for column in data_encoded.columns:
    if column == 'revenue':
        continue
    correlation, p_value = pearsonr(data_encoded[column], data_encoded['revenue'])
    correlations[column] = correlation
    p_values[column] = p_value


In [56]:
average_correlation = sum(correlations.values())
average_correlation / (len(correlations)) 

-0.0011096810650601365

In [57]:
max(correlations.values())

0.2232284506897029

In [58]:
filtered_correlations = {key: value for key, value in correlations.items() if value > 0.6}

In [59]:
filtered_correlations

{}

In [53]:
# Step 1: Split the list of categorical values into separate rows
data_expanded = o_df.explode('spoken_languages_id')

# Step 2: Create dummy variables for each unique category
dummy_variables = pd.get_dummies(data_expanded['spoken_languages_id'], prefix='spoken_langauges_id')

# Step 3: Aggregate the dummy variables
aggregated_dummy = dummy_variables.groupby(level=0).sum()

# Concatenate the aggregated dummy variables with the original DataFrame
data_encoded = pd.concat([o_df[['revenue']], aggregated_dummy], axis=1)
# o_df['encoded_spoken_languages'] = aggregated_dummy.apply(lambda x: np.array(x), axis=1)

data_encoded.head()

,revenue,spoken_langauges_id_af,spoken_langauges_id_am,spoken_langauges_id_ar,spoken_langauges_id_bg,spoken_langauges_id_bm,spoken_langauges_id_bn,spoken_langauges_id_bo,spoken_langauges_id_br,spoken_langauges_id_bs,...,spoken_langauges_id_tr,spoken_langauges_id_uk,spoken_langauges_id_ur,spoken_langauges_id_vi,spoken_langauges_id_wo,spoken_langauges_id_xh,spoken_langauges_id_xx,spoken_langauges_id_yi,spoken_langauges_id_zh,spoken_langauges_id_zu
0,2787965087,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,961000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,880674609,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1084939099,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,284139100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
from scipy.stats import pearsonr
correlations = {}
p_values = {}
for column in data_encoded.columns:
    if column == 'revenue':
        continue
    correlation, p_value = pearsonr(data_encoded[column], data_encoded['revenue'])
    correlations[column] = correlation
    p_values[column] = p_value


In [48]:
average_correlation = sum(correlations.values())
average_correlation / (len(correlations)) 

0.005965953968371287

In [49]:
max(correlations.values())

0.09356865617796768

In [50]:
filtered_correlations = {key: value for key, value in correlations.items() if value > 0.6}

In [51]:
filtered_correlations

{}